In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras import models,layers
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.preprocessing import LabelEncoder

In [3]:
def getDataSize(path):
  image_count = {}

  for x in os.listdir(path):
    image_count[x] = len(os.listdir(os.path.join(path, x)))

  return image_count

In [4]:
train_path = '/content/drive/MyDrive/chest_ct_scans/train'
test_path = '/content/drive/MyDrive/chest_ct_scans/test'
valid_path = '/content/drive/MyDrive/chest_ct_scans/valid'

In [5]:
batch_size = 8

In [6]:
def generateImage(path):
  generate_data = ImageDataGenerator(rescale=1.0/255.0,
                                horizontal_flip=True,
                                fill_mode='nearest',
                                zoom_range=0.2,
                                shear_range=0.2,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                rotation_range=30)

  return generate_data.flow_from_directory(path, batch_size=batch_size, target_size=(256, 256), color_mode='grayscale', class_mode='categorical')

In [7]:
def generateImageRGB(path):
  generate_data = ImageDataGenerator(rescale=1.0/255.0,
                                horizontal_flip=True,
                                fill_mode='nearest',
                                zoom_range=0.2,
                                shear_range=0.2,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                rotation_range=30)

  return generate_data.flow_from_directory(path, batch_size=batch_size, target_size=(256, 256), class_mode='categorical')

In [8]:
train_data = generateImage(train_path)
test_data = generateImage(test_path)
valid_data = generateImage(valid_path)

Found 986 images belonging to 5 classes.
Found 150 images belonging to 5 classes.
Found 132 images belonging to 5 classes.


In [14]:
number_of_classes = len(train_data.class_indices)

## Basic Sequential Model

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, GlobalAveragePooling2D
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3),activation='relu',input_shape=(256,256,1)))

model.add(Conv2D(filters=64, kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Flatten())

model.add(Dense(units=64,activation='relu'))
model.add(Dropout(rate=0.50))

model.add(Dense(units=number_of_classes,activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 254, 254, 32)      320       
                                                                 
 conv2d_5 (Conv2D)           (None, 252, 252, 64)      18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 126, 126, 64)     0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 126, 126, 64)      0         
                                                                 
 flatten_2 (Flatten)         (None, 1016064)           0         
                                                                 
 dense_3 (Dense)             (None, 64)                65028160  
                                                      

In [18]:
model.fit(x=train_data, steps_per_epoch=8, epochs=64, validation_data=test_data, validation_steps=8)

Epoch 1/64
8/8 [==============================] - 61s 7s/step - loss: 6.7910 - accuracy: 0.2812 - val_loss: 1.8110 - val_accuracy: 0.2812
Epoch 2/64
8/8 [==============================] - 53s 7s/step - loss: 1.5400 - accuracy: 0.3438 - val_loss: 1.4937 - val_accuracy: 0.2812
Epoch 3/64
8/8 [==============================] - 47s 6s/step - loss: 1.5750 - accuracy: 0.3750 - val_loss: 1.3159 - val_accuracy: 0.4844
Epoch 4/64
8/8 [==============================] - 42s 5s/step - loss: 1.3230 - accuracy: 0.5000 - val_loss: 1.3080 - val_accuracy: 0.4375
Epoch 5/64
8/8 [==============================] - 40s 5s/step - loss: 1.4055 - accuracy: 0.4688 - val_loss: 1.3505 - val_accuracy: 0.3906
Epoch 6/64
8/8 [==============================] - 42s 5s/step - loss: 1.4227 - accuracy: 0.4375 - val_loss: 1.2824 - val_accuracy: 0.4062
Epoch 7/64
8/8 [==============================] - 36s 5s/step - loss: 1.2833 - accuracy: 0.5781 - val_loss: 1.3804 - val_accuracy: 0.3750
Epoch 8/64
8/8 [==================

## MobileNet

In [19]:
train_data = generateImageRGB(train_path)
test_data = generateImageRGB(test_path)
valid_data = generateImageRGB(valid_path)

Found 986 images belonging to 5 classes.
Found 150 images belonging to 5 classes.
Found 132 images belonging to 5 classes.


In [20]:
from keras.applications import MobileNet
from keras.models import Model

base_model = MobileNet(input_shape=(256, 256, 3), weights='imagenet', include_top=False)

for layer in base_model.layers:
  layer.trainable = False  # Freeze the pre-trained layers

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(512,activation='relu')(x) #dense layer 2
x=Dense(128,activation='relu')(x) #dense layer 3
preds=Dense(5,activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=base_model.input,outputs=preds)

17225924/17225924 [==============================] - 0s 0us/step


In [21]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 128, 128, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 128, 128, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 128, 128, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 128, 128, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 128, 128, 32)     128       
 ation)                                                      

In [22]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer

model.fit(x=train_data,steps_per_epoch=8, epochs = 64, validation_data=test_data, validation_steps=test_data.samples//batch_size)

Epoch 1/64
8/8 [==============================] - 21s 2s/step - loss: 1.9092 - accuracy: 0.3281 - val_loss: 1.5119 - val_accuracy: 0.4722
Epoch 2/64
8/8 [==============================] - 18s 2s/step - loss: 1.3520 - accuracy: 0.4688 - val_loss: 0.9180 - val_accuracy: 0.6528
Epoch 3/64
8/8 [==============================] - 25s 4s/step - loss: 1.0829 - accuracy: 0.5469 - val_loss: 0.9591 - val_accuracy: 0.5417
Epoch 4/64
8/8 [==============================] - 17s 2s/step - loss: 0.9227 - accuracy: 0.6379 - val_loss: 0.8936 - val_accuracy: 0.6181
Epoch 5/64
8/8 [==============================] - 25s 4s/step - loss: 1.0307 - accuracy: 0.5000 - val_loss: 0.9424 - val_accuracy: 0.6111
Epoch 6/64
8/8 [==============================] - 17s 2s/step - loss: 0.8185 - accuracy: 0.5938 - val_loss: 0.8122 - val_accuracy: 0.5903
Epoch 7/64
8/8 [==============================] - 18s 2s/step - loss: 0.8824 - accuracy: 0.5312 - val_loss: 0.8025 - val_accuracy: 0.5764
Epoch 8/64
8/8 [==================